In [1]:
import pandas as pd

df = pd.read_csv("dataset.csv")

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os.path

chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

homedir = os.path.expanduser("~")
chrome_options.binary_location = f"{homedir}/chrome-linux64/chrome"
webdriver_service = Service(f"{homedir}/chromedriver-linux64/chromedriver")

driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [3]:
import ast
from selenium.webdriver.remote.webelement import WebElement

import re

def keep_assignments(code_snippet):
    # Regex to match variable assignments. This pattern assumes variable names are valid Python identifiers
    # and captures typical assignment statements, excluding those that might appear in comments or strings.
    pattern = r'^\s*[a-zA-Z_][a-zA-Z0-9_]*\s*=\s*.+'

    # Filter and keep only lines with variable assignments
    assignments = [line for line in code_snippet.split('\n') if re.match(pattern, line)]

    # Join the filtered lines back into a string
    return "\n".join(assignments)

# This function will be used to visit each node in the AST
class VariableVisitor(ast.NodeVisitor):
    def __init__(self):
        super().__init__()
        self.output = []
    
    def visit_Assign(self, node):
        
        # For each assignment, print the targets (variables)
        for target in node.targets:
            if isinstance(target, ast.Name):  # Ensure it's a variable assignment
                self.output.append(target.id)
def get_web_elements(code_snippet):
    # Parse the code snippet into an AST
    parsed_code = ast.parse(code_snippet)

    # Create a visitor instance and use it to visit the nodes in the parsed AST
    visitor = VariableVisitor()
    visitor.visit(parsed_code)
    variables = visitor.output

    output = []
    variable_type = WebElement
    for var_name in variables:
            # Use globals() to get the variable from its name since we are in the global scope
        var_value = globals().get(var_name, None)
        
        # Check if the variable exists and its type
        if var_value is not None and isinstance(var_value, variable_type):
            output.append(var_value)
    return output

In [4]:
from tqdm import tqdm

def load_html(html, driver):
    file_path = 'sample_page.html'

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html)
        
    abs_file_path = os.path.abspath("sample_page.html")

    # Use the file:/// protocol to load the local HTML file
    driver.get(f"file:///{abs_file_path}")

output = []

for index, row in tqdm(df.iterrows()):
    html = row['html']
    load_html(html, driver)

    code = row['code']
    code = keep_assignments(code)

    exec(code)

    variables = get_web_elements(code)

    # We assume there is only one variable defined
    variable = variables[0]
    outer_html = variable.get_attribute("outerHTML")

    output.append(outer_html)
    
print(output)

0it [00:00, ?it/s]

10it [02:16, 13.62s/it]

['<a class="btn-mktg home-campaign-enterprise btn-muted-mktg" data-analytics-event="{&quot;category&quot;:&quot;Start a free enterprise trial&quot;,&quot;action&quot;:&quot;click to Start a free enterprise trial&quot;,&quot;label&quot;:&quot;ref_cta:Start a free enterprise trial;&quot;}" href="/organizations/enterprise_plan?ref_cta=Start+a+free+enterprise+trial&amp;ref_loc=Home+campaign+hero&amp;ref_page=%2F" data-test-selector="start-trial-button">\n  Start a free enterprise trial\n  \n  <svg xmlns="http://www.w3.org/2000/svg" class="octicon arrow-symbol-mktg" width="16" height="16" viewBox="0 0 16 16" fill="none"><path fill="currentColor" d="M7.28033 3.21967C6.98744 2.92678 6.51256 2.92678 6.21967 3.21967C5.92678 3.51256 5.92678 3.98744 6.21967 4.28033L7.28033 3.21967ZM11 8L11.5303 8.53033C11.8232 8.23744 11.8232 7.76256 11.5303 7.46967L11 8ZM6.21967 11.7197C5.92678 12.0126 5.92678 12.4874 6.21967 12.7803C6.51256 13.0732 6.98744 13.0732 7.28033 12.7803L6.21967 11.7197ZM6.21967 4.2803

In [5]:

from lavague.action_engine import ActionEngine
from lavague.defaults import DefaultEmbedder

import os
from llama_index.llms.azure_openai import AzureOpenAI

from lavague.prompts import DEFAULT_PROMPT

api_key=os.getenv("AZURE_OPENAI_KEY")
api_version="2023-05-15"
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model = "gpt-4"
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4-turbo")

class LLM(AzureOpenAI):
    def __init__(self):
        super().__init__(
            model=model,
            deployment_name=deployment_name,
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            temperature=0.0
        )
llm = LLM()
embedder = DefaultEmbedder()

action_engine = ActionEngine(llm, embedder, streaming=False)

/home/daniel/miniconda3/envs/lavague/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 743/743 [00:00<00:00, 5.15MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:00<00:00, 527MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 2.10MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 50.4MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 40.1MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 1.02MB/s]


In [6]:
from lavague.prompts import DEFAULT_PROMPT

In [7]:
row

url                                https://www.hubspot.com
html     <html lang="en"><head>\n    <meta charset="utf...
query                            click on get started free
node     <a class="\n  cl-button -secondary -large wf-p...
code     # Let's proceed step by step.\n# First we need...
Name: 9, dtype: object

In [12]:
from lavague.action_engine import extract_first_python_code

scores = []
generated_codes = []
generated_outer_htmls = []

for index, row in tqdm(df.iterrows()):
    node = row['node']
    query = row['query']

    prompt = DEFAULT_PROMPT.format(context_str=node, query_str=query)
    generated_code = llm.complete(prompt).text
    generated_code = extract_first_python_code(generated_code)

    load_html(html, driver)

    generated_code = keep_assignments(generated_code)
    generated_codes.append(generated_code)
    try:
        exec(generated_code)
        variables = get_web_elements(generated_code)

        # We assume there is only one variable defined
        variable = variables[0]
        outer_html = variable.get_attribute("outerHTML")
        generated_outer_htmls.append(outer_html)

        if outer_html == output[index]:
            scores.append(1)
        else:
            scores.append(0)
    except Exception as e:
        scores.append(0)
        print("Error at index: ", index)
        print("Non executable code")
        print("Code snippet: ", generated_code)
        print("Error: ", e)

1it [00:30, 30.67s/it]

Error at index:  0
Non executable code
Code snippet:  trial_link = driver.find_element(By.LINK_TEXT, "Start a free enterprise trial")
Error:  Message: no such element: Unable to locate element: {"method":"link text","selector":"Start a free enterprise trial"}
  (Session info: chrome-headless-shell=122.0.6261.94); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x555a35c02e93 <unknown>
#1 0x555a358face6 <unknown>
#2 0x555a35945e48 <unknown>
#3 0x555a35945f01 <unknown>
#4 0x555a359893f4 <unknown>
#5 0x555a35967edd <unknown>
#6 0x555a35986899 <unknown>
#7 0x555a35967c53 <unknown>
#8 0x555a35938db3 <unknown>
#9 0x555a3593977e <unknown>
#10 0x555a35bc87cb <unknown>
#11 0x555a35bcc7e5 <unknown>
#12 0x555a35bb60e1 <unknown>
#13 0x555a35bcd372 <unknown>
#14 0x555a35b9a1bf <unknown>
#15 0x555a35bf1488 <unknown>
#16 0x555a35bf1683 <unknown>
#17 0x555a35c02044 <unknown>
#18 0x7f387b0976

2it [00:42, 19.79s/it]

Error at index:  1
Non executable code
Code snippet:  datasets_link = driver.find_element(By.LINK_TEXT, "Datasets")
Error:  Message: no such element: Unable to locate element: {"method":"link text","selector":"Datasets"}
  (Session info: chrome-headless-shell=122.0.6261.94); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x555a35c02e93 <unknown>
#1 0x555a358face6 <unknown>
#2 0x555a35945e48 <unknown>
#3 0x555a35945f01 <unknown>
#4 0x555a359893f4 <unknown>
#5 0x555a35967edd <unknown>
#6 0x555a35986899 <unknown>
#7 0x555a35967c53 <unknown>
#8 0x555a35938db3 <unknown>
#9 0x555a3593977e <unknown>
#10 0x555a35bc87cb <unknown>
#11 0x555a35bcc7e5 <unknown>
#12 0x555a35bb60e1 <unknown>
#13 0x555a35bcd372 <unknown>
#14 0x555a35b9a1bf <unknown>
#15 0x555a35bf1488 <unknown>
#16 0x555a35bf1683 <unknown>
#17 0x555a35c02044 <unknown>
#18 0x7f387b097609 start_thread



3it [01:06, 21.77s/it]

...

Error at index:  5
Non executable code
Code snippet:  gmail_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Gmail')]")
Error:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(text(), 'Gmail')]"}
  (Session info: chrome-headless-shell=122.0.6261.94); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x555a35c02e93 <unknown>
#1 0x555a358face6 <unknown>
#2 0x555a35945e48 <unknown>
#3 0x555a35945f01 <unknown>
#4 0x555a359893f4 <unknown>
#5 0x555a35967edd <unknown>
#6 0x555a35986899 <unknown>
#7 0x555a35967c53 <unknown>
#8 0x555a35938db3 <unknown>
#9 0x555a3593977e <unknown>
#10 0x555a35bc87cb <unknown>
#11 0x555a35bcc7e5 <unknown>
#12 0x555a35bb60e1 <unknown>
#13 0x555a35bcd372 <unknown>
#14 0x555a35b9a1bf <unknown>
#15 0x555a35bf1488 <unknown>
#16 0x555a35bf1683 <unknown>
#17 0x555a35c02044 <unknown>
#18 0x7f387b097609 sta

7it [01:51, 12.38s/it]

Error at index:  6
Non executable code
Code snippet:  free_trial_link = driver.find_element(By.LINK_TEXT, "Start free trial")
Error:  Message: no such element: Unable to locate element: {"method":"link text","selector":"Start free trial"}
  (Session info: chrome-headless-shell=122.0.6261.94); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x555a35c02e93 <unknown>
#1 0x555a358face6 <unknown>
#2 0x555a35945e48 <unknown>
#3 0x555a35945f01 <unknown>
#4 0x555a359893f4 <unknown>
#5 0x555a35967edd <unknown>
#6 0x555a35986899 <unknown>
#7 0x555a35967c53 <unknown>
#8 0x555a35938db3 <unknown>
#9 0x555a3593977e <unknown>
#10 0x555a35bc87cb <unknown>
#11 0x555a35bcc7e5 <unknown>
#12 0x555a35bb60e1 <unknown>
#13 0x555a35bcd372 <unknown>
#14 0x555a35b9a1bf <unknown>
#15 0x555a35bf1488 <unknown>
#16 0x555a35bf1683 <unknown>
#17 0x555a35c02044 <unknown>
#18 0x7f387b097609 start_thread



7it [01:53, 16.25s/it]


KeyboardInterrupt: 

In [17]:
scores = []
generated_codes = []

generated_outer_htmls = []

for index, row in tqdm(df.iterrows()):
    html = row['html']
    query = row['query']

    generated_code, retrieved_nodes = action_engine.get_action(query, html)

    load_html(html, driver)

    generated_code = keep_assignments(generated_code)
    generated_codes.append(generated_code)
    try:
        exec(generated_code)
        variables = get_web_elements(generated_code)

        # We assume there is only one variable defined
        variable = variables[0]
        outer_html = variable.get_attribute("outerHTML")
        generated_outer_htmls.append(outer_html)

        if outer_html == output[index]:
            scores.append(1)
        else:
            scores.append(0)
    except Exception as e:
        generated_outer_htmls.append("None")
        scores.append(0)
        print("Error at index: ", index)
        print("Non executable code")
        print("Code snippet: ", generated_code)
        print("Error: ", e)

2it [00:29, 15.61s/it]

Error at index:  1
Non executable code
Code snippet:  dataset_links = driver.find_elements(By.XPATH, "//a[contains(@class, 'block p-2')]")
Error:  list index out of range


10it [02:34, 15.48s/it]


In [16]:
generated_outer_htmls

['<a class="btn-mktg home-campaign-enterprise btn-muted-mktg" data-analytics-event="{&quot;category&quot;:&quot;Start a free enterprise trial&quot;,&quot;action&quot;:&quot;click to Start a free enterprise trial&quot;,&quot;label&quot;:&quot;ref_cta:Start a free enterprise trial;&quot;}" href="/organizations/enterprise_plan?ref_cta=Start+a+free+enterprise+trial&amp;ref_loc=Home+campaign+hero&amp;ref_page=%2F" data-test-selector="start-trial-button">\n  Start a free enterprise trial\n  \n  <svg xmlns="http://www.w3.org/2000/svg" class="octicon arrow-symbol-mktg" width="16" height="16" viewBox="0 0 16 16" fill="none"><path fill="currentColor" d="M7.28033 3.21967C6.98744 2.92678 6.51256 2.92678 6.21967 3.21967C5.92678 3.51256 5.92678 3.98744 6.21967 4.28033L7.28033 3.21967ZM11 8L11.5303 8.53033C11.8232 8.23744 11.8232 7.76256 11.5303 7.46967L11 8ZM6.21967 11.7197C5.92678 12.0126 5.92678 12.4874 6.21967 12.7803C6.51256 13.0732 6.98744 13.0732 7.28033 12.7803L6.21967 11.7197ZM6.21967 4.2803

In [18]:
df["scores"] = scores
df["generated_codes"] = generated_codes
df["generated_outer_htmls"] = generated_outer_htmls

df.to_csv("dataset_with_scores.csv", index=False)

In [21]:
from IPython.display import HTML, display

for i, row in df.loc[df["scores"] == 0].iterrows():
    code = row["code"]
    gold_html = output[i]
    generated_code = row["generated_codes"]
    generated_html = row["generated_outer_htmls"]
    print("Gold standard code: ", code)
    display(HTML(gold_html))
    print(gold_html)
    print("Generated code: ", generated_code)
    display(HTML(generated_html))
    print(generated_html)
    print("\n\n\n")

Gold standard code:  # Let's proceed step by step.
# First we need to identify the link first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the link.
# The link can be identified using the text "Datasets"
# We can use the XPATH to select the link
datasets_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Datasets')]")

# Then we can click on it
datasets_link.click()


<a class="group flex items-center px-2 py-0.5 dark:hover:text-gray-400 hover:text-red-700" href="/datasets"><svg class="mr-1.5 text-gray-400 group-hover:text-red-500" style="" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" aria-hidden="true" focusable="false" role="img" width="1em" height="1em" preserveAspectRatio="xMidYMid meet" viewBox="0 0 25 25"><ellipse cx="12.5" cy="5" fill="currentColor" fill-opacity="0.25" rx="7.5" ry="2"></ellipse><path d="M12.5 15C16.6421 15 20 14.1046 20 13V20C20 21.1046 16.6421 22 12.5 22C8.35786 22 5 21.1046 5 20V13C5 14.1046 8.35786 15 12.5 15Z" fill="currentColor" opacity="0.5"></path><path d="M12.5 7C16.6421 7 20 6.10457 20 5V11.5C20 12.6046 16.6421 13.5 12.5 13.5C8.35786 13.5 5 12.6046 5 11.5V5C5 6.10457 8.35786 7 12.5 7Z" fill="currentColor" opacity="0.5"></path><path d="M5.23628 12C5.08204 12.1598 5 12.8273 5 13C5 14.1046 8.35786 15 12.5 15C16.6421 15 20 14.1046 20 13C20 12.8273 19.918 12.1598 19.7637 12C18.9311 12.8626

None




Gold standard code:  # Let's proceed step by step.
# First we need to identify the link first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the link.
# The competitions link can be identified using the class name "sc-gjLLEI sc-fAscey kIbkrq dDqYED" and the text "Competitions"
competitions_link = driver.find_element(By.XPATH, "//*[contains(text(), 'Competitions')]")

# Then we can click on it
competitions_link.click()


<a href="/competitions" data-click-log-id="nav-con-competitions" class="sc-jtMbrs bwioyi">Competitions</a>
Generated code:  competitions_link = driver.find_element(By.XPATH, "//a[@href='/docs/competitions']")


<a href="/docs/competitions" data-click-log-id="km-footer-docs-competitions" class="sc-gjLLEI sc-fAscey kIbkrq dDqYED">Competitions</a>




Gold standard code:  # Let's proceed step by step.
# First we need to identify the button first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the button.
# The action button can be identified using the ID "pt-createaccount-2"
action_button = driver.find_element(By.XPATH, "//*[@id='pt-createaccount-2']")

# Then we can click on it
action_button.click()


...

<a href="/w/index.php?title=Special:CreateAccount&amp;returnto=Main+Page" title="You are encouraged to create an account and log in; however, it is not mandatory"><span class="vector-icon mw-ui-icon-userAdd mw-ui-icon-wikimedia-userAdd"></span> <span>Create account</span></a>




Gold standard code:  # Let's proceed step by step.
# First we need to identify the search bar first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the search bar.
# The search bar can be identified using the class name "index-search__input"
search_bar = driver.find_element(By.XPATH, "//*[@class='index-search__input']")

# Then we can click on it
search_bar.click()


<div class="index-search__input"><input type="text" class="index-search__input-value index-search__input-value--xl svelte-ambpd9" aria-label="Enter domain, keyword or URL" placeholder="Enter domain, keyword or URL" role="combobox" aria-autocomplete="list" aria-controls="suggest-list" aria-expanded="false"> <div class="index-search__input-outline  svelte-ambpd9"></div></div>
Generated code:  search_bar = driver.find_element(By.XPATH, "//input[@placeholder='Enter domain, keyword or URL']")


<input type="text" class="index-search__input-value index-search__input-value--xl svelte-ambpd9" aria-label="Enter domain, keyword or URL" placeholder="Enter domain, keyword or URL" role="combobox" aria-autocomplete="list" aria-controls="suggest-list" aria-expanded="false">




Gold standard code:  # Let's proceed step by step.
# First we need to identify the button first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the button.
# The action button can be identified using the class name "cl-button -secondary -large wf-page-header__cta homepage-hero2-cta"
# We can use the contains function to select the button
get_started_free_button = driver.find_element(By.XPATH, "//*[contains(@class, 'cl-button -secondary -large wf-page-header__cta homepage-hero2-cta')]")

# Then we can click on it
get_started_free_button.click()


<a class="
  cl-button -secondary -large wf-page-header__cta homepage-hero2-cta" href="https://app.hubspot.com/signup-hubspot/crm?hubs_signup-url=www.hubspot.com&amp;hubs_signup-cta=homepage-hero2-cta&amp;__hstc=753710.2dd10f2fc1a8042f42261521b6eda287.1711594168300.1711594168300.1711594168300.1&amp;__hssc=753710.1.1711594168300&amp;__hsfp=3584728142" rel="noreferrer " data-hs-event-96027165="1">
        Get started free
        
          <span class="visually-hidden">with HubSpot's free tools</span>
        
      </a>
Generated code:  get_started_free_button = driver.find_element(By.XPATH, "//*[contains(@class, 'homepage-platform-free') or contains(@class, 'homepage-products-free')]")


<a class="cl-button -secondary -medium homepage-platform-free" href="https://app.hubspot.com/signup-hubspot/crm?hubs_signup-cta=homepage-platform-free&amp;hubs_signup-url=www.hubspot.com&amp;__hstc=753710.9541084b8df4e80fbb678f928a90f60e.1711595731173.1711595731173.1711595731173.1&amp;__hssc=753710.1.1711595731173&amp;__hsfp=3584728142" rel="noreferrer " data-hs-event-96027165="1">
              Get started free
              
                <span class="visually-hidden">with HubSpot's free tools</span>
              
            </a>






In [11]:
import numpy as np

accuracy = np.mean(scores)
accuracy

0.5

In [12]:
from IPython.display import HTML, display

for html in output:
    display(HTML(html))